In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk
import logging
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
a = torch.rand(1,10)
a


In [ ]:
b = F.softmax(a,dim=1)
b,b.argmax(dim=1)

In [ ]:
a = [1,2,3,8,9,11]
b = [4,5,6,10]

la = len(a)
lb = len(b)

max_ = max(la,lb)
for k in range(max_):
    pair_train = a[k]
    pair_test = b[k%lb]
    print(pair_train,pair_test)

In [4]:
model__ = torch.load("cnn_extract_model.pth")

In [8]:
a = torch.rand(1,3,224,224)


In [ ]:
model__["state_dict"]

In [2]:

def model():

    model = models.resnet34(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    p = 0
    for child in model.children():
        if p >= 6:
            for param in child.parameters():
                param.requires_grad = True
        p += 1

    return model


class RESNET(nn.Module):
    def __init__(self, original_model):
        super(RESNET, self).__init__()
        self.feature_extractor = nn.Sequential(*list(original_model.children())[:-1])
        self.fc1 = nn.Linear(in_features=512,out_features=99,bias=True)
        #self.fc2 = nn.Linear(in_features=512,out_features=99,bias=True)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        (m,nc,nw,nh) = x.size()
        cnn = self.feature_extractor(x).view(m,-1)
        fc1 = self.fc1(cnn)
        #relu1 = F.relu(fc1)
        drop1 = self.dropout(fc1)
        #fc2 = self.fc2(drop1)
        out = F.log_softmax(drop1,dim=1)
        return out

In [6]:
m = model()
res = RESNET(m)

res.load_state_dict(torch.load("cnn_extract_model.pth")["state_dict"])


In [ ]:
res(a)

In [ ]:
train = "Frames/Train/"

def data_loader(path,batch_size,shuffle):
    all_img_list = []
    all_class_list = []
    
    train_list = glob.glob(path+"*")
    
    class_no = 0
    for classes in train_list:
        class_list = glob.glob(classes+"/*")
        for folders in class_list:
            img_list = glob.glob(folders+"/*.jpg")
            for images in img_list:
                all_img_list.append(images)
                all_class_list.append(class_no)
        
        class_no += 1
    
    index = np.random.permutation(len(all_img_list))
    random_img = np.array(all_img_list)[index]
    random_class = np.array(all_class_list)[index]
    
    for i in range(0,len(all_img_list),batch_size):
        yield(random_img[i:i+batch_size],random_class[i:i+batch_size])
                


In [64]:
def prepare_data(im_list,cls_list):
    X = []
    Y = []
    for im,cl in zip(im_list,cls_list):
        image = Image.open(im)
        im_array = preprocess(image).numpy()
        X.append(im_array)
        Y.append(cl)
    return np.array(X),np.array(Y)
        

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

sub_batches = data_loader(train,16,True)
for imgs,clls in sub_batches:
    X,Y = prepare_data(imgs,clls)
    print(X.shape,Y)
    break
    

(16, 3, 224, 224) [66 59 48 55 13 16 72 91 74 75 40 74 34 97 75 94]


In [51]:
a = np.random.permutation(4)
a

array([2, 1, 0, 3])

In [52]:
x = np.array(["a","b",5,6])


In [53]:
x[a]

array(['5', 'b', 'a', '6'], 
      dtype='<U1')

In [61]:
a = ["k","as","df"]
b = [4,5,6]
for i,j in zip(a,b):
    print(i,j)

k 4
as 5
df 6
